In [ ]:
!pip install googlesearch-python
!pip install googletrans
!pip install gtts
!pip install deep_translator
import os
import time
import openai
import requests
import gradio as gr
from bs4 import BeautifulSoup
from googletrans import Translator
from deep_translator import GoogleTranslator
from gtts import gTTS
import tempfile
from googlesearch import search
from urllib.parse import urlparse
from dotenv import load_dotenv
from openai import OpenAI


In [67]:
load_dotenv(override=True)
openai_api_key=os.getenv('OPENAI_API_KEY')
                         
if openai_api_key:
    print(f'your api is ready and starts with {openai_api_key[:8]}')               
                         
                    

your api is ready and starts with sk-proj-


In [68]:
openai=OpenAI()
MODEL="gpt-4o-mini"

In [137]:
#scrap the website content
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
def scrape_website(url):
    try:
        res = requests.get(url, timeout=10, headers=headers)
        res.raise_for_status()  
        soup = BeautifulSoup(res.text, 'html.parser')
        for element in soup(['script', 'style', 'nav', 'footer', 'aside', 'noscript', 'iframe', 'header']):
            element.decompose()
        for element in soup.find_all():
            if len(element.get_text(strip=True)) == 0:
                element.decompose()
        text = soup.get_text(separator=' ', strip=True)
        return text[:5000]
    except Exception as e:
        print(f"Error: {e}") 
        return "unable to retrieve content from website"


In [138]:
def is_valid_url(url):
    """Check if a URL is valid and properly formatted"""
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

In [135]:
def info_google_search(query, num_results=7):
    try:
        results = []
        for result in search(query, num_results=num_results, advanced=True):
            if not is_valid_url(result.url):
                print(f"Skipping invalid URL: {result.url}")
                continue
                
            data = {
                "title": result.title,
                "url": result.url,
                "description": result.description,
                "scraped_content": scrape_website(result.url) if is_valid_url(result.url) else "Invalid URL"
            }
            results.append(data)
            time.sleep(2)
        return results
    except Exception as e:
        print(f"Error: {e}")
        return []

In [128]:
#Build a system prompt
def build_prompt(company_name, website_text, lang_name, search_results):
    return f"""
You are {company_name}'s AI assistant, responding in {lang_name}. Combine official company information with 
external insights to provide comprehensive, accurate responses.

**Knowledge Base:**
🔹 **Official Company Information:**
{website_text}

🔹 **Market Context (Top 7 Sources):**
{search_results}

**Response Requirements:**
✓ Adapt communication style for {lang_name} cultural norms
✓ Synthesize information from both sources when possible
✓ Highlight official information when conflicts exist
✓ Use markdown formatting for clarity (bullet points, bold key terms)
✓provide links for information you provided if possible.
✓respond in same way like some of the top llms like llama,gemini,openai responds


"""

In [117]:
translator=Translator()


In [118]:
def translate(text,target_lang):
    try:
        return GoogleTranslator(source='auto',target=target_lang).translate.text
    except:
        return text
    
   

In [119]:
def speak(text,lang_code='en'):
    tts=gTTS(text=text,lang=lang_code)
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts.save(fp.name)
        return fp.name

In [120]:
lang_map = {
    "English": "en",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Hindi": "hi",
    "Arabic": "ar",
    "Chinese (Simplified)": "zh-cn"
}


In [121]:
#main
def chat(company_name,company_url,user_lang_label,user_input):
    lang_code=lang_map[user_lang_label]
    website_text=scrape_website(company_url)
    results=info_google_search(company_name)
    prompt=build_prompt(company_name,website_text,user_lang_label,results)
    translated_input = translate(user_input, 'en') if lang_code != 'en' else user_input
    messages = [{"role": "system", "content": prompt},
                {"role": "user", "content": translated_input}]
    
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    reply_en = response.choices[0].message.content.strip()  
 # Translate back to user language
    reply_user_lang =translate(reply_en, lang_code) if lang_code != 'en' else reply_en
    audio_path = speak(reply_user_lang, lang_code)

    return reply_user_lang, audio_path
    

In [143]:
iface = gr.Interface(
    fn=chat,
    
    inputs=[
        gr.Text(label="Company Name"),
        gr.Text(label="Company Website URL"),
        gr.Dropdown(label="Preferred Language", choices=list(lang_map.keys()), value="English"),
        gr.Textbox(label="Your Question")
    ],
    outputs=[
        gr.Textbox(label="Bot Response"),
        gr.Audio(label="Voice Output")
    ],
    title="Voice-Activated Information Service for Multiple Languages",
    description="Custom chatbot for any company. Just input company name, website, and your preferred language."
)

iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7880

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
